In [1]:
import numpy as np
import pandas as pd
import yaml
import cv2
import os
import glob
import xml.etree.ElementTree as ET

In [2]:
root_folder = '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated'
clip_file_list = glob.glob(root_folder + '/**/by_key/video=*/clip=*', recursive=True)
print(len(clip_file_list), 'clips found.')

117 clips found.


In [3]:
clip_file_list[:10]

['/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated/object-detection/by_key/video=98/clip=165434_170826_4',
 '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated/object-detection/by_key/video=248/clip=136663_142055_4',
 '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated/object-detection/by_key/video=184/clip=145484_150876_4',
 '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated/object-detection/by_key/video=313/clip=85716_91108_4',
 '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated/object-detection/by_key/video=263/clip=96504_101900_4',
 '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/videos/annotated/object-detection/by_key/video=125/clip=86313_91705_4',
 '/home/golden/server-red_see/annotation_exports/data/annotation-project-export/vi

In [16]:
for clip_file in clip_file_list:
    yaml_file = os.path.join(clip_file, 'info.yaml')
    with open(yaml_file, 'r') as f:
        yaml_df = pd.io.json.json_normalize(yaml.load(f))
    print('clip_file', clip_file)
    print('clip_name', yaml_df.loc[0,'original_video.filename'])
    print('-----------------')

clip_file /home/golden/server-kenya_drones/loopy_annotations/Ungulate_Tracking/videos/annotated/object-detection/by_key/video=981/clip=0_1500_15
clip_name NOV08_2017_DJI_0118.MOV
-----------------
clip_file /home/golden/server-kenya_drones/loopy_annotations/Ungulate_Tracking/videos/annotated/object-detection/by_key/video=1043/clip=0_1499_15
clip_name 000002215.mp4
-----------------
clip_file /home/golden/server-kenya_drones/loopy_annotations/Ungulate_Tracking/videos/annotated/object-detection/by_key/video=1044/clip=0_1499_15
clip_name 000002216.mp4
-----------------
clip_file /home/golden/server-kenya_drones/loopy_annotations/Ungulate_Tracking/videos/annotated/object-detection/by_key/video=1018/clip=0_1499_15
clip_name 000002181.mp4
-----------------
clip_file /home/golden/server-kenya_drones/loopy_annotations/Ungulate_Tracking/videos/annotated/object-detection/by_key/video=1020/clip=0_1499_15
clip_name 000002183.mp4
-----------------
clip_file /home/golden/server-kenya_drones/loopy_an

In [ ]:
pd.options.display.max_columns = 100


output_folder = '/media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/' 

video_root_folder =  '/home/golden/server-red_see/Original_files' 
 

annotations_folder_path = output_folder + 'annotations/'

if not os.path.exists(annotations_folder_path):
    os.makedirs(annotations_folder_path)


for clip_file in clip_file_list:
    yaml_file = os.path.join(clip_file, 'info.yaml')
    with open(yaml_file, 'r') as f:
        yaml_df = pd.io.json.json_normalize(yaml.load(f))
    video_name = yaml_df.loc[0,'original_video.filename']
    try:
        video_file = glob.glob(video_root_folder + '/**/' + video_name, recursive=True)[0]
#         print(video_file)
    except Exception as e:
        print('video_name', video_name)
        print('video_file',clip_file)
        print(e)
        continue
    annotation_df = pd.read_feather(os.path.join(clip_file, 'annotations.feather'))
    frames_list = pd.read_csv(os.path.join(clip_file, 'frames-framesnum.csv')) .loc[:, 'frame_num']
    cap = cv2.VideoCapture(video_file)
    if cap is None or not cap.isOpened():
        raise Exception('Opencv unable to open the video file for %s' %video_file)
        
    video_name = video_name.split('.')[0]
    
    for frame_num, frame in enumerate(frames_list):
        cap.set(getattr(cv2, 'CAP_PROP_POS_FRAMES', 1), frame)
        _, image = cap.read()
        frame_name = video_name + '-' + str(frame) 
        frame_file = output_folder + frame_name + '.jpg'
        cv2.imwrite(frame_file, image)
        
        
        new_xml = ET.Element('annotation')
        folder = ET.SubElement(new_xml, 'folder')
        folder.text = 'annotated-frames'
        filename = ET.SubElement(new_xml, 'filename')
        filename.text = frame_name + '.jpg'
        image_size = ET.SubElement(new_xml, 'size')
        image_width = ET.SubElement(image_size, 'width')
        image_width.text = str(image.shape[1])
        image_height = ET.SubElement(image_size, 'height')
        image_height.text = str(image.shape[0])
        image_depth = ET.SubElement(image_size, 'depth')
        image_depth.text = str(image.shape[2])

        segmented = ET.SubElement(new_xml, 'segmented')
        segmented.text = '0'
        for row in annotation_df.index[annotation_df.loc[:, 'frame_num'] == frame_num].tolist():
            animal_object = ET.SubElement(new_xml, 'object')
            #name
            name = annotation_df.loc[row, 'name']
            name = name.split('_')
            animal_name = name[0]
#             animal_age = name[1]
            name = ET.SubElement(animal_object, 'name')
            name.text = animal_name
            #pose
            pose = ET.SubElement(animal_object, 'pose')
            pose.text = 'Unspecified'
            #truncated
            truncated = ET.SubElement(animal_object, 'truncated')
            truncated.text = '0'
            #difficult
            difficult = ET.SubElement(animal_object, 'difficult')
            difficult.text = '0'

#             polygon_list = animal.findall('polygon')
#             points = polygon_list[frame].findall('pt')
#             top_left = [(points[0].find('x').text), (points[0].find('y').text)]
#             bottom_right = [(points[2].find('x').text), (points[2].find('y').text)]

            #bndbox
            bndbox = ET.SubElement(animal_object, 'bndbox')
            xmin = ET.SubElement(bndbox, 'xmin')
            xmin.text = str(annotation_df.loc[row, 'min_col'])
            ymin = ET.SubElement(bndbox, 'ymin')
            ymin.text = str(annotation_df.loc[row, 'min_row'])
            xmax = ET.SubElement(bndbox, 'xmax')
            xmax.text = str(annotation_df.loc[row, 'max_col'])
            ymax = ET.SubElement(bndbox, 'ymax')
            ymax.text = str(annotation_df.loc[row, 'max_row'])

            #age
#             age = ET.SubElement(animal_object, 'age')
#             age.text = animal_age 
        outfile = output_folder + 'annotations/' + frame_name + str('.xml')
        print('outfile', outfile)
        tree = ET.ElementTree(new_xml)
        tree.write(outfile)
        

cap.release()
        

        
        
        





outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165434.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165438.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165442.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165446.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165450.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165454.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165458.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165462.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-frames-xml/annotations/C0037-165466.xml
outfile /media/golden/72FFC6EE48B5CF39/server-red_see/annotated-

In [5]:
file = '/home/golden/server-kenya_drones/loopy_annotations/v1/videos/annotated/object-detection/by_key/video=830/clip=12000_13500_15/annotations.feather' 
df = pd.read_feather(file)
df.head()

,auto_obj_id,obj_id,frame_num,manual,min_col,min_row,max_col,max_row,name,origin
0,0,ID_0,0,True,1907,564,2007,646,buffalo_adult,b'human'
1,0,ID_0,1,False,1899,559,1999,641,buffalo_adult,b'human'
2,0,ID_0,2,False,1889,554,1989,636,buffalo_adult,b'human'
3,0,ID_0,3,False,1878,549,1978,631,buffalo_adult,b'human'
4,0,ID_0,4,False,1870,544,1970,626,buffalo_adult,b'human'


In [22]:
yaml_file = '/Volumes/kenya_drones/loopy_annotations/v1/videos/annotated/object-detection/by_key/video=830/clip=12000_13500_15/info.yaml'
with open(yaml_file, 'r') as f:
    yaml_df = pd.io.json.json_normalize(yaml.load(f))
pd.options.display.max_columns = 100
print(pd.options.display.max_columns)

yaml_df

100


,annotation_id,annotation_project.classes.,annotation_project.classes.PZ_adult,annotation_project.classes.PZ_foal,annotation_project.classes.buffalo_adult,annotation_project.classes.buffalo_calf,annotation_project.description,annotation_project.id,annotation_project.mode,annotation_project.title,annotation_project.user.email,annotation_project.user.id,annotations.Buffalo.annotator,annotations.Buffalo.date,annotations.Buffalo.df,annotations.Buffalo.path,annotations.Buffalo.type,annotations.Buffalo.xml,collections,end_frame,id,original_video.codec,original_video.duration,original_video.extension,original_video.filename,original_video.fps,original_video.frame0,original_video.frame_count,original_video.frame_max,original_video.framerate,original_video.height,original_video.id,original_video.local_path,original_video.md5,original_video.parent_id,original_video.pix_fmt,original_video.ready,original_video.recorded_date,original_video.size,original_video.user_id,original_video.uuid,original_video.version,original_video.video_type,original_video.width,parent_id,remap_frames,segment.end_frame,segment.original_fps,segment.start_frame,segment.step,start_frame,step,video.codec,video.fps,video.height,video.length_f,video.path,video.pixel_fmt,video.seekable,video.transcoded,video.user_description,video.user_title,video.width,video_id
0,321,#ff33ff,#00c1fb,#031fa0,#00ff33,#47803c,Buffalo drone footage,8,rectangle,Buffalo,blair.costelloe@orn.mpg.de,7,7,2018-01-18 08:43:43.511048,annotations.feather,None,labelme,annotations.xml,[],13500,"[video_id, start_frame, end_frame, step]",h264,327.460467,.MOV,NOV02_2017_DJI_0018.MOV,59.94006,0,19628,19627,60000/1001,2160,830,None,,None,yuv420p,True,None,4092508487,7,89790e6a33e9348e25cd4ed4158d0e45,None,0,4096,[video_id],True,13500,59.94006,12000,15,12000,15,h264,3.996004,2160,100,/home/loopy/loopb.io/data/videos/0/30/830/0000...,yuv420p,None,None,None,None,4096,830


In [24]:
annotation_df.index[annotation_df.loc[:, 'frame_num'] == frame_num].tolist()


[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300]

In [32]:
video_file = '' 

# video_stream = cv2.VideoCapture(video_file)
# total_frames = video_stream.get(cv2.CAP_PROP_FRAME_COUNT)

annotated_frames = df.loc[:, 'frame_num'] .unique()

for frame in range(total_frames):
    if frame == annotated_frame
    video_stream.read()


print(annotated_frames[0])

0
